<a href="https://colab.research.google.com/github/robert-mdh-bui/uge_deicing_heatwatch_2022/blob/main/folium-dashboard-scratchbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install airportsdata
!pip install dash
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import airportsdata as ad

import haversine
from haversine import haversine, haversine_vector, Unit

import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
import folium
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt
from folium import plugins

#from google.colab import data_table
#data_table.enable_dataframe_formatter()


In [ ]:
# Reading publicly-available airport metadata
airports = pd.DataFrame.from_dict(ad.load('IATA'),orient = 'index')[['iata','name','country','lat','lon']]
airports = airports[airports.country == "US"]

In [ ]:
# Reading GHCND Station Metadata
colspecs = [(0, 11), (12, 20), (21, 30), (31, 37), (38, 40), (41, 71), (73,-1)]

ghcnd_metadata = pd.read_fwf("/content/drive/MyDrive/career/United Ground/uge_deicing_heatwatch_2022/data_noaa/ghcnd-stations.txt",
                             colspecs = colspecs)
ghcnd_metadata = ghcnd_metadata.drop(ghcnd_metadata.columns[[4,6]],axis=1)

In [ ]:
# Extracting monthly normals data (to filter out stations with missing data)
mly = pd.read_csv('/content/drive/MyDrive/career/United Ground/uge_deicing_heatwatch_2022/data_noaa/by_variable/mly-normal-allall.csv')

df_stations = mly[['GHCN_ID']]\
  .drop_duplicates()\
  .rename(columns = {'GHCN_ID' : "id"})\
  .merge(ghcnd_metadata,
         on = "id",
         how = 'left')

In [ ]:
df_stations['zipped'] = list(zip(df_stations.lat,df_stations.lon))
airports['zipped']    = list(zip(airports.lat,   airports.lon))

In [413]:
# Calculating pairwise distances (haversine - not taking altitude into account)
distances = pd.DataFrame(haversine_vector(airports.zipped.tolist(),df_stations.zipped.tolist(),Unit.MILES, comb=True))
distances.columns = airports.iata.tolist()
distances.index = df_stations.id.tolist()
# Saving dataframes to .pkl file
distances.to_pickle('distances.pkl')
airports.to_pickle('airports.pkl')
df_stations.to_pickle('df_stations.pkl')

In [ ]:
# Draw map of all stations
mp_allstations = folium.Map(location=[39.50,-98.5795], zoom_start=4, control_scale=True)

for i in range(0,len(df_stations)):
  folium.Marker(
      [df_stations.iloc[i]['lat'],df_stations.iloc[i]['lon']],
      popup = df_stations.iloc[i]['id'] + "\n" + df_stations.iloc[i]['name']
  ).add_to(mp_allstations)

mp_allstations

In [ ]:
# Finding nearest 5 NOAA stations to input airport
# Working example here is ORD - O'Hare International

def draw_top5_map(input_iata):
  top5stations = distances[[input_iata]]\
    .sort_values(by = input_iata)\
    .head(5)\
    .reset_index()\
    .merge(df_stations,
          left_on='index',
          right_on = 'id',
          how = 'left')\
    .rename(columns={input_iata:'distance'})

  input_lat = float(airports[airports.iata == input_iata].lat)
  input_lon = float(airports[airports.iata == input_iata].lon)

  # Init folium map
  m5 = folium.Map(location=[input_lat,input_lon], 
                  zoom_start=10, 
                  control_scale=True,
                  tiles="Stamen Terrain")

  # Add root marker for airport
  folium.Marker(
      [input_lat,input_lon],
      popup = input_iata,
      icon = folium.Icon(color='blue',
                        icon='plane',
                        prefix='fa')
  ).add_to(m5)

  # Add NOAA station markers
  for i in range(0,len(top5stations)):
    folium.Marker(
        [top5stations.iloc[i]['lat'],top5stations.iloc[i]['lon']],
        popup = top5stations.iloc[i]['id'] + "\n" + top5stations.iloc[i]['name'] + "\n" + str(top5stations.iloc[i]['distance']),
        icon = folium.Icon(color='green',
                        icon='cloud',
                        prefix='fa')
    ).add_to(m5)

  # Init + add minimap
  minimap = plugins.MiniMap()
  m5.add_child(minimap)

  return(m5)

draw_top5_map("ORD")